## 590 Project：Doodle Image Recognition Using LSTM+CNN model
## Yarou Xu, Fengdi Li, Yifan Wu

## 1. Set up working environment

In [3]:
!pip install  keras==2.1.0
import keras
keras.__version__

    100% |████████████████████████████████| 307kB 9.2MB/s 
textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.0 which is incompatible.
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


'2.2.4'

In [4]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 324.2 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## 2. Preprocessing the quick draw data:
     a. Filter sketching drawn by Chinese and American.
     b. Filter sketchings that has been successfully recognized.

In [0]:
import os
import pandas as pd
# Check all the sketching categories
path = '/content/drive/My Drive/quick_draw_training/'
files = os.listdir(path)
files
    

['bread.csv',
 'pliers.csv',
 'remote control.csv',
 'pickup truck.csv',
 'envelope.csv',
 'popsicle.csv',
 'sea turtle.csv',
 'elephant.csv',
 'castle.csv',
 'ambulance.csv',
 'river.csv',
 'bandage.csv',
 'nail.csv',
 'line.csv',
 'bucket.csv',
 'bus.csv',
 'cello.csv',
 'ocean.csv',
 'truck.csv',
 'camouflage.csv',
 'harp.csv',
 'telephone.csv',
 'stairs.csv',
 'star.csv',
 'guitar.csv',
 'sandwich.csv',
 'sun.csv',
 'feather.csv',
 'leaf.csv',
 'toilet.csv',
 'strawberry.csv',
 'waterslide.csv',
 'bottlecap.csv',
 'coffee cup.csv',
 'banana.csv',
 'dresser.csv',
 'house plant.csv',
 'skateboard.csv',
 'skyscraper.csv',
 'pizza.csv',
 'hammer.csv',
 'teapot.csv',
 'giraffe.csv',
 'underwear.csv',
 'snowman.csv',
 'monkey.csv',
 'computer.csv',
 'pencil.csv',
 'shovel.csv',
 'necklace.csv',
 'compass.csv',
 'bat.csv',
 'bicycle.csv',
 'teddy-bear.csv',
 'scorpion.csv',
 'hot dog.csv',
 'fish.csv',
 'see saw.csv',
 'rain.csv',
 'snail.csv',
 'sink.csv',
 'belt.csv',
 'speedboat.csv',


In [0]:
len(files)

340

Because the US sketching data is too big for the google colab to process, we select the top 100 and top 1000 images per category.

In [0]:
import numpy
# Filter the dataset so that the cleaned dataset only contains sketching 
# which are successfully recognized and from US or China.
df = pd.read_csv(path + files[0])
df = pd.read_csv(path + files[0])
df['recognized'] = df['recognized'].astype(bool)
df = df[(df['countrycode'] == 'US') & (df['recognized'] == True)]
df = df[:1000]

i = 1
for file_i in files[1:]:
  df_i = pd.read_csv(path + file_i)
  df_i['recognized'] = df_i['recognized'].astype(bool)
  df_i = df_i[(df_i['countrycode'] == 'US') & (df_i['recognized'] == True)]
  df_i = df_i[:1000]

  df = df.append(df_i)
  print(file_i + ' finished, ' + str(len(files) - i) + ' to go.')
  i += 1  

pliers.csv finished, 339 to go.
remote control.csv finished, 338 to go.
pickup truck.csv finished, 337 to go.
envelope.csv finished, 336 to go.
popsicle.csv finished, 335 to go.
sea turtle.csv finished, 334 to go.
elephant.csv finished, 333 to go.
castle.csv finished, 332 to go.
ambulance.csv finished, 331 to go.
river.csv finished, 330 to go.
bandage.csv finished, 329 to go.
nail.csv finished, 328 to go.
line.csv finished, 327 to go.
bucket.csv finished, 326 to go.
bus.csv finished, 325 to go.
cello.csv finished, 324 to go.
ocean.csv finished, 323 to go.
truck.csv finished, 322 to go.
camouflage.csv finished, 321 to go.
harp.csv finished, 320 to go.
telephone.csv finished, 319 to go.
stairs.csv finished, 318 to go.
star.csv finished, 317 to go.
guitar.csv finished, 316 to go.
sandwich.csv finished, 315 to go.
sun.csv finished, 314 to go.
feather.csv finished, 313 to go.
leaf.csv finished, 312 to go.
toilet.csv finished, 311 to go.
strawberry.csv finished, 310 to go.
waterslide.csv fin

In [0]:
# Save cleaned dataset
df.to_csv('us_all_1000.csv')
!cp './us_all_1000.csv' '/content/drive/My Drive/590-cleaned-data/'

## 3. Convert raw strokes into images

In [0]:
# Define a function to convert raw strokes into images with 256 by 256 pixels
def draw_cv2(raw_strokes, size=256, lw=3):   
    step = len(raw_strokes)
    max_x = max( [max(x)  for x, y in raw_strokes ] )
    max_y = max( [max(y)  for x, y in raw_strokes ] )   
    max_x, max_y =  max_x + 1, max_y + 1
    img = np.zeros((max_y, max_x), np.uint8)   
    for i in range(step):
        stroke = raw_strokes[i]        
        x, y=stroke[0], stroke[1]
        for i in range(len(x) - 1):
            _ = cv2.line(img, (x[i], y[i]),(x[i + 1], y[i + 1]), 255, lw)   
    top = max ((size- max_y) // 2 , 0 )
    bottom = max ( size- top - max_y , 0)
    left = max ((size - max_x) // 2, 0 )
    right = max (size - max_x - left, 0 )
    img =  cv2.copyMakeBorder(img,top,bottom,left,right,cv2.BORDER_CONSTANT)
    return img

In [8]:
# Prepare training and validate dataset for the LSTM+CNN model
import pandas as pd
import ast
import cv2
from keras.utils import np_utils
import numpy as np
df= pd.read_csv('/content/drive/My Drive/590-cleaned-data/us_all_1000.csv')
X_train = []
y_train = pd.Categorical(df['word'], categories=df['word'].unique()).codes
df['word'].nunique()
Y_train = np_utils.to_categorical(y_train, 340)
df['drawing'] = df['drawing'].map(ast.literal_eval)

# Due to computation and memory limitation, we need to shrink the size of the original 256 by 256 images
for i in range(len(df)):
    image = draw_cv2(df['drawing'][i], size=256, lw=3)
    # Shrink the size into 28 by 28
    image = cv2.resize(image, (28,28), interpolation = cv2.INTER_CUBIC)
    X_train.append(image)
    
# Convert the dataset into 5 dimensions for the LSTM+CNN model
X_train = np.array(X_train).reshape(len(X_train), 1, 28, 28,1).astype('float32') / 255
Y_train = Y_train.reshape(len(Y_train), 1,340)
print(X_train.shape)
print(Y_train.shape)

# Split into training and validate datasets
from sklearn.model_selection import train_test_split
train_X, valid_X, train_Y, valid_Y = train_test_split(X_train,Y_train,test_size=0.2, random_state=13)

(341000, 1, 28, 28, 1)
(341000, 1, 340)


In [0]:
# Prepare training and validate dataset for the LSTM+CNN model
import pandas as pd
import ast
import cv2
from keras.utils import np_utils
import numpy as np
df= pd.read_csv('/content/drive/My Drive/590-cleaned-data/us_all_1000.csv')
X_train = []
y_train = pd.Categorical(df['word'], categories=df['word'].unique()).codes
df['word'].nunique()
Y_train = np_utils.to_categorical(y_train, 340)
df['drawing'] = df['drawing'].map(ast.literal_eval)

# Due to computation and memory limitation, we need to shrink the size of the original 256 by 256 images
for i in range(len(df)):
    image = draw_cv2(df['drawing'][i], size=256, lw=3)
    # Shrink the size into 28 by 28
    image = cv2.resize(image, (28,28), interpolation = cv2.INTER_CUBIC)
    X_train.append(image)
    
# Convert the dataset into 4 dimensions for the basic CNN model    
X_train = np.array(X_train).reshape(len(X_train),28,28,1).astype('float32') / 255
print(X_train.shape)
print(Y_train.shape)

# Split into training and validate datasets
from sklearn.model_selection import train_test_split
train_X, valid_X, train_Y, valid_Y = train_test_split(X_train,Y_train,test_size=0.2, random_state=13)

(341000, 28, 28, 1)
(341000, 340)


In [1]:
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import SimpleRNN, LSTM, Embedding, GRU
from keras.layers import TimeDistributed, Lambda
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model, Sequential
from keras.preprocessing import sequence

C:\Users\47532\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


## 4. The basic CNN model

In [2]:
# Basic CNN model: 2 Conv2D layers, 1 maxpooling layer, 2 drop out layers and 2 full connection layers
model = Sequential()
model.add(Conv2D(filters = 32,kernel_size=(3, 3), 
                 activation='relu', strides=(1, 1), 
                 padding='valid',
                 input_shape=(28,28,1)))
model.add(Conv2D(filters = 32,kernel_size=(3, 3), 
                 activation='relu', strides=(1, 1), 
                 padding='valid'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(340, activation='softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1179904   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Training the CNN model
model.fit(train_X, train_Y,
         epochs = 80,
         batch_size = 1024,
        validation_data=(valid_X, valid_Y))

Train on 272800 samples, validate on 68200 samples
Epoch 1/80
272800/272800 [==============================] - 22s 79us/step - loss: 5.0143 - acc: 0.0545 - val_loss: 3.8024 - val_acc: 0.2352
Epoch 2/80
272800/272800 [==============================] - 18s 68us/step - loss: 3.9794 - acc: 0.1601 - val_loss: 3.2089 - val_acc: 0.3136
Epoch 3/80
272800/272800 [==============================] - 19s 69us/step - loss: 3.6528 - acc: 0.2070 - val_loss: 2.9792 - val_acc: 0.3479
Epoch 4/80
272800/272800 [==============================] - 19s 70us/step - loss: 3.5004 - acc: 0.2293 - val_loss: 2.8427 - val_acc: 0.3721
Epoch 5/80
272800/272800 [==============================] - 19s 70us/step - loss: 3.4039 - acc: 0.2452 - val_loss: 2.7759 - val_acc: 0.3834
Epoch 6/80
272800/272800 [==============================] - 19s 68us/step - loss: 3.3443 - acc: 0.2538 - val_loss: 2.7277 - val_acc: 0.3935
Epoch 7/80
272800/272800 [==============================] - 19s 68us/step - loss: 3.2979 - acc: 0.2616 - val_

KeyboardInterrupt: ignored

## 5. The LSTM+CNN model

In [4]:
# The LSTM+CNN model: 2 Conv2D layers, 1 maxpooling layer, 1 dropout layer, 1 LSTM layer and 2 full connection layers
model = Sequential()
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu'),
                          input_shape=(None, 28, 28, 1)))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))
model.add(TimeDistributed(Conv2D(32, (3,3), activation='relu')))
model.add(TimeDistributed(Dropout(0.8)))
# Convert the 2D ouput into one sequential vector as input of the LSTM layer
model.add(TimeDistributed(Flatten()))
# define LSTM model
model.add(LSTM(128, return_sequences=True))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(340, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, None, 26, 26, 64)  640       
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 25, 25, 64)  0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 23, 23, 32)  18464     
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 23, 23, 32)  0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 16928)       0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         8733184   
_________________________________________________________________
dense_3 (Dense)              (None, None, 256)         33024     
__________

In [5]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [0]:
# Training the LSTM+CNN model
model.fit(train_X, train_Y,
         epochs = 60,
         batch_size = 1024,
        validation_data=(valid_X, valid_Y))

Train on 272800 samples, validate on 68200 samples
Epoch 1/60
272800/272800 [==============================] - 50s 182us/step - loss: 4.1019 - acc: 0.1621 - val_loss: 3.1080 - val_acc: 0.2994
Epoch 2/60
272800/272800 [==============================] - 46s 167us/step - loss: 2.9691 - acc: 0.3198 - val_loss: 2.6984 - val_acc: 0.3670
Epoch 3/60
272800/272800 [==============================] - 46s 167us/step - loss: 2.6517 - acc: 0.3733 - val_loss: 2.4429 - val_acc: 0.4159
Epoch 4/60
272800/272800 [==============================] - 45s 166us/step - loss: 2.4692 - acc: 0.4061 - val_loss: 2.3285 - val_acc: 0.4336
Epoch 5/60
272800/272800 [==============================] - 46s 167us/step - loss: 2.3439 - acc: 0.4291 - val_loss: 2.2508 - val_acc: 0.4478
Epoch 6/60
272800/272800 [==============================] - 46s 168us/step - loss: 2.2475 - acc: 0.4470 - val_loss: 2.1913 - val_acc: 0.4614
Epoch 7/60
272800/272800 [==============================] - 46s 168us/step - loss: 2.1640 - acc: 0.4634

## 6. Use the test data to test the performance of  the basic CNN model and LSTM+CNN model. In this section, we does cross test here which means that when training on US dataset, then we will test the model on Chinese dataset , and vice versa.

In [10]:
# Read in the test dataset for the LSTM+CNN model
# When training on US data, the Chinese sketching data is test data
df_cn = pd.read_csv('/content/drive/My Drive/590-cleaned-data/df_cn.csv')
X_test = []
y_test = pd.Categorical(df_cn['word'], categories=df['word'].unique()).codes
Y_test = np_utils.to_categorical(y_test, 340)
df_cn['drawing'] = df_cn['drawing'].map(ast.literal_eval)
for i in range(len(df_cn)):
    image = draw_cv2(df_cn['drawing'][i], size=256, lw=3)
    image = cv2.resize(image, (28,28), interpolation = cv2.INTER_CUBIC)
    X_test.append(image)
X_test = np.array(X_test).reshape(len(X_test), 1, 28,28,1).astype('float32') / 255
Y_test = Y_test.reshape(len(Y_test), 1,340)
print(X_test.shape)
print(y_test.shape)
print(Y_test.shape)

(5138, 1, 28, 28, 1)
(5138,)
(5138, 1, 340)


In [0]:
# Read in the test dataset for the CNN model
# When training on US data, the Chinese sketching data is test data
import pandas as pd
import ast
import cv2
from keras.utils import np_utils
import numpy as np
df_cn = pd.read_csv('/content/drive/My Drive/590-cleaned-data/df_cn.csv')
X_test = []
y_test = pd.Categorical(df_cn['word'], categories=df['word'].unique()).codes
Y_test = np_utils.to_categorical(y_test, 340)
df_cn['drawing'] = df_cn['drawing'].map(ast.literal_eval)
for i in range(len(df_cn)):
    image = draw_cv2(df_cn['drawing'][i], size=256, lw=3)
    image = cv2.resize(image, (28,28), interpolation = cv2.INTER_CUBIC)
    X_test.append(image)
X_test = np.array(X_test).reshape(len(X_test), 28, 28,1).astype('float32') / 255
#Y_test = Y_test.reshape(len(Y_test), 1,340)
print(X_test.shape)
print(y_test.shape)
print(Y_test.shape)

(5138, 28, 28, 1)
(5138,)
(5138, 340)


In [0]:
score = model.evaluate(X_test, Y_test, verbose = 1)
print(score)

5138/5138 [==============================] - 1s 190us/step
[2.616116162170553, 0.42604126119112495]
